In [1]:
import ee
import datetime
import os
import itertools
import sys

from pprint import pprint
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE

In [3]:
ee.Initialize()

### Define basic parameters

In [142]:
# define the periods to perform temporal check
Check_len = 3
Check_wieght = [3,1,1]
Check_threshold = 3

# define the number of iteration
Check_iteration_num = 10

In [143]:
# define the names of each period
year_range = list(f'{i[0]}_{i[1]}' for i in zip(range(1990,2018,3),range(1992,2020,3)))

In [144]:
year_range

['1990_1992',
 '1993_1995',
 '1996_1998',
 '1999_2001',
 '2002_2004',
 '2005_2007',
 '2008_2010',
 '2011_2013',
 '2014_2016',
 '2017_2019']

### Import Classified_random_img

In [141]:
# get the classified_random_imgs and sum them up for each period
Classified_imgs = [ee.ImageCollection(f"users/wangjinzhulala/North_China_Plain_Python/classification_img/Random_{year}")
                     .sum()
                     .gte(8)
                     .set('name',year)
                  for year in year_range]

In [152]:
def Temporal_check(in_tifs,weights,threshold):

    # zip classified_random_sum with weights
    img_multiply = list(zip(in_tifs,weights))

    # multiply each classified_sum_img with check_weight 
    # and sum 3-periods together
    sum_tif = ee.ImageCollection([i[0].multiply(i[1]) for i in img_multiply]).sum()
    
    # thoes pixel that greater than threshold will be kept
    sum_constrained = sum_tif.gt(threshold)
    
    return sum_constrained

In [153]:
Iter_tempral_check_instaces = {}

for it in range(Check_iteration_num):
    
    if it == 0:
        chuncks   = [Classified_imgs[i:i+3] for i in range(len(Classified_imgs)-2)]
        check_tif = [Temporal_check(chunk,Check_wieght,Check_threshold) for chunk in chuncks]
        Iter_tempral_check_instaces[0]  = check_tif
        
    else:
        in_tifs   = Iter_tempral_check_instaces[0] + Classified_imgs[8:10]
        chuncks   = [in_tifs[i:i+3] for i in range(len(Classified_imgs)-2)]
        
        check_tif = [Temporal_check(chunk,Check_wieght,Check_threshold) for chunk in chuncks]
        Iter_tempral_check_instaces[it] = check_tif

In [154]:
Iter_tempral_check_instaces

{0: [<ee.image.Image at 0x29eab952198>,
 1: [<ee.image.Image at 0x29eab53d550>,
 2: [<ee.image.Image at 0x29eab40ce80>,
 3: [<ee.image.Image at 0x29eab9108d0>,
 4: [<ee.image.Image at 0x29eafb59da0>,
 5: [<ee.image.Image at 0x29eac059320>,
 6: [<ee.image.Image at 0x29eac5f42b0>,
 7: [<ee.image.Image at 0x29eac070390>,
 8: [<ee.image.Image at 0x29eabb0bac8>,
 9: [<ee.image.Image at 0x29eabd15748>,
  <ee.image.Image at 0x29eab8ce9b0>]}

In [163]:
Map = geemap.Map()
Map.setCenter(115.4508, 35.2492,10)

Map.add_basemap('HYBRID')
Map.addLayer(Classified_imgs[9] ,{'min':0,'max':1},'origin')
Map.addLayer(Iter_tempral_check_instaces[0][1] ,{'min':0,'max':1},'Iter_0')
Map.addLayer(Iter_tempral_check_instaces[3][1] ,{'min':0,'max':1},'Iter_3')
Map.addLayer(Iter_tempral_check_instaces[5][1] ,{'min':0,'max':1},'Iter_5')
Map.addLayer(Iter_tempral_check_instaces[9][1] ,{'min':0,'max':1},'Iter_9')
Map

Map(center=[35.2492, 115.4508], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButt…